# Fine tuning BERT base model from HuggingFace on Amazon SageMaker

## Overview

This tutotial uses the Hugging Face transformers and datasets libraries with Amazon SageMaker to fine-tune a pre-trained BERT base model on binary text classification.

A [pre-trained model](https://huggingface.co/bert-base-uncased) is available in the transformers library from Hugging Face. You’ll be fine-tuning this pre-trained model using the [Amazon Reviews dataset](https://huggingface.co/datasets/amazon_reviews_multi) and classify the review into either positive or negative feedback.

This Jupyter Notebook can run on any type of SageMaker Notebook instances with kernel (e.g. `ml.t3.medium` with `conda_pytorch_p38`). You can set up your SageMaker Notebook instance by following the [Get Started with Amazon SageMaker Notebook Instances](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-console.html) documentation.

## Development Environment and Permissions

For SageMaker Studio, please make sure ipywidgets is installed and restart the kernel

In [ ]:
%%capture
import IPython
import sys

!{sys.executable} -m pip install ipywidgets
IPython.Application.instance().kernel.do_shutdown(True)  # has to restart kernel so changes are used

This tutorial requires the following pip packages:
  + transformers
  + datasets
 
Please make sure SageMaker version is 2.116.0 or byond.

In [ ]:
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade --no-cache-dir 'sagemaker>=2.116.0'
!{sys.executable} -m pip install --upgrade --no-cache-dir 'boto3>=1.26.8' 'botocore>=1.29.12' s3fs
!{sys.executable} -m pip install --upgrade --no-cache-dir torch==1.11.0
!{sys.executable} -m pip install --upgrade --no-cache-dir transformers==4.16.2 datasets==2.5.2

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
from transformers import AutoTokenizer
from datasets import load_dataset

### Create Sagemaker session
Next, create a SageMaker session and define an execution role. Default role should suffice.

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
sess_bucket = sess.default_bucket()

print(f'sagemaker role arn: {role}')
print(f'sagemaker bucket: {sess_bucket}')
print(f'sagemaker session region: {sess.boto_region_name}')

## Dataset preparation
For this example, we will use the `datasets` library to download and preprocess the `amazon_review_multi` dataset from Hugging Face. In original dataset the review sentences are classified as 5 Ratings. We will treat it as binary classification task where the review sentences are classified as Positive or Negative. We will use the original dataset of 5 and 4 as Positive (LABEL_1), 2 and 1 as Negative (LABEL_0), and data with Ratings of 3 will not be used.

In [ ]:
# pre-trained model used
model_name = 'bert-base-uncased'

# dataset used
dataset_name = 'amazon_reviews_multi'

# s3 key prefix
s3_prefix = 'bert-base-uncased-amazon-reviews'

In [ ]:
# Prepare dataset
dataset = load_dataset(dataset_name, 'en')
dataset = dataset.remove_columns(['review_id', 'product_id', 'reviewer_id', 'review_title', 'language', 'product_category'])
dataset = dataset.filter(lambda dataset: dataset['stars'] != 3)
dataset = dataset.map(lambda dataset: {'labels': int(dataset['stars'] > 3)}, remove_columns=['stars'])

# Tokenization
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples['review_body'], padding='max_length', max_length=128, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['review_body'])
tokenized_datasets.set_format('torch')

# Reduced the dataset for testing purpose
train_dataset = tokenized_datasets['train'].shuffle().select(range(4000))
eval_dataset = tokenized_datasets['test'].shuffle().select(range(256))

Let's take a look one example to understand how original sentence is tokeninzed and encoded.

In [ ]:
index = 100000
print(dataset['train'][index])
print('Tokenize:', tokenizer.tokenize(dataset['train']['review_body'][index]))
print('Encode:', tokenizer.encode(dataset['train']['review_body'][index]))

### Uploading dataset to S3 Bucket
After we processed the dataset we are going to use the new FileSystem integration to upload our dataset to S3.

In [ ]:
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

# save dataset to s3
training_input_path = f's3://{sess_bucket}/{s3_prefix}/{dataset_name}/train'
train_dataset.save_to_disk(training_input_path, fs=s3)

test_input_path = f's3://{sess_bucket}/{s3_prefix}/{dataset_name}/test'
eval_dataset.save_to_disk(test_input_path, fs=s3)

## Leveraging Neuron Persistent Cache
PyTorch Neuron performs just-in-time compilation of graphs during execution. At every step if the traced graph varies from the previous executions, it is compiled by the neuron compiler.
The compilation result can be saved in on-disk [Neuron Persistent Cache](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/arch/neuron-features/neuron-caching.html) to avoid compilations across training runs. In order to keep such cache across SageMaker traing jobs, we add the mechanism to restore the cache from S3 at the begging of the training job and then store back to S3 at the end of the training job.

In [ ]:
import pathlib

# If neuron compile cache does not exist in S3, create dummy file.
if not sess.list_s3_files(sess_bucket, f'{s3_prefix}/neuron-compile-cache.tar.gz'):
    pathlib.Path('./dummy').touch()
    with tarfile.open('neuron-compile-cache.tar.gz', 'w:gz') as t:
        t.add('dummy')
    sess.upload_data('./neuron-compile-cache.tar.gz', sess_bucket, s3_prefix)

s3_prefix_path = f's3://{sess_bucket}/{s3_prefix}'

## Fine-tuning & starting Sagemaker Training Job
In order to create a sagemaker training job we use [PyTorch Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#train-a-model-with-pytorch). The Estimator handles end-to-end Amazon SageMaker training and deployment tasks. In a Estimator we define, which fine-tuning script should be used as `entry_point`, which `instance_type` should be used, which `hyperparameters` are passed in .....

Note) SageMaker support for EC2 Trn1 instance is currently available only for PyTorch Estimator. HuggingFace Estimator will be available in future release.

In [ ]:
# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 10,               # number of training epochs
                 'train_batch_size': 8,      # batch size for training
                 'eval_batch_size': 8,       # batch size for evaluation
                 'learning_rate': 5e-5,      # learning rate used during training
                 'model_name':model_name,    # pre-trained model
                }

In [ ]:
pt_estimator = PyTorch(
    entry_point='train.py',
    source_dir='scripts',
    role=role,
    instance_count=1,
    instance_type='ml.trn1.2xlarge',
    framework_version='1.11.0',
    py_version='py38',
    base_job_name=s3_prefix,
    hyperparameters=hyperparameters,

    distribution={
        'torch_distributed': {
            'enabled': True
        }
    }
)

In [ ]:
!pygmentize ./scripts/train.py

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {
    'train': training_input_path,
    'test': test_input_path,
    's3_prefix': s3_prefix_path  # To restore Neuron Persistent Cache archive from S3
}

In [ ]:
%%time
# starting the train job with our uploaded datasets as input
pt_estimator.fit(data, wait=True)

### Backup Neuron Persistent Cache into S3 for the following training jobs

In [ ]:
TrainingJobName = pt_estimator.latest_training_job.describe()['TrainingJobName']

sess.download_data('./', sess_bucket, f'{TrainingJobName}/output/output.tar.gz')

import tarfile
with tarfile.open('output.tar.gz', 'r') as t:
    t.extractall('./output/')

with tarfile.open('neuron-compile-cache.tar.gz', 'w:gz') as t:
    t.add('./output/neuron-compile-cache', arcname='neuron-compile-cache')

sess.upload_data('./neuron-compile-cache.tar.gz', sess_bucket, s3_prefix)

## Quick check the prediction result.
Let's quickly check to see if the fine-tuned model produces the expected prediction results.


In [ ]:
sess.download_data('./', sess_bucket, f'{TrainingJobName}/output/model.tar.gz')
    
with tarfile.open('model.tar.gz', 'r') as t:
    t.extractall('./model/')

In [ ]:
from transformers import pipeline

classifier = pipeline('text-classification', model = './model/')

print(classifier('This is the most amazing product I have ever seen. I love it.'))
print(classifier('big disappointment. I will not use it.'))
print(classifier('Wonderful product. I will let my friends know.'))

### Congratulation ! You can see the expected results !